https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

In [1]:
import pandas as pd

In [2]:
# for windows using Path
from pathlib import Path
data_folder = Path("C:\\Users\\User\\Dropbox\\python-pandas-code\\bayes")
# Note two backslashes for Windows

In [14]:
# For Mac
bdi_in = pd.read_csv("/Users/arvindkarir/Dropbox/python-pandas-code/bayes/data_bdi.csv")
fedrate_in = pd.read_csv("/Users/arvindkarir/Dropbox/python-pandas-code/bayes/data_fedrate.csv")
yield_in = pd.read_csv("/Users/arvindkarir/Dropbox/python-pandas-code/bayes/data_10y3mo.csv")
# Getting path from Terminal adds '\' to spaces, needs to be removed

In [189]:
# bring in the bdi data
#bdi_in = pd.read_csv(data_folder / "data_bdi.csv")

In [15]:
df1 = pd.DataFrame(bdi_in)
df2 = pd.DataFrame(fedrate_in)
df3 = pd.DataFrame(yield_in)

In [29]:
df1['year'] = pd.to_datetime(df1.date).dt.year
df1['qtr'] = pd.to_datetime(df1.date).dt.quarter

In [30]:
mean_bdi = df1.groupby(['year', 'qtr'])['bdi'].mean()

In [31]:
# export data for merge later
df1.to_csv(r'data_bdi_fixed.csv', columns = ['year','qtr','bdi','r'] ,index=False)

# bring in fed rates file
fedrate_in = pd.read_csv(data_folder / "data_fedrate.csv")

In [23]:
# convert date to year and quarter
df2['year'] = pd.to_datetime(df2.date).dt.year
df2['qtr'] = pd.to_datetime(df2.date).dt.quarter

In [33]:
# need to find the mean rate for each quarter
mean_fedrate = df2.groupby(['year', 'qtr'])['fed_rate'].mean()

#twomean_fedrate = df2.groupby(['year', 'qtr']).agg( ['mean']) # results in a multi-index column
#mean_fedrate.columns = mean_fedrate.columns.droplevel(1)

In [34]:
mean_fedrate[:10]

year  qtr
1954  3      1.026667
      4      0.986667
1955  1      1.343333
      2      1.500000
      3      1.940000
      4      2.356667
1956  1      2.483333
      2      2.693333
      3      2.810000
      4      2.926667
Name: fed_rate, dtype: float64

In [36]:
mean_fedrate.to_csv(r'data_fedrate_fixed.csv', header = "True")
# adding header = "True" took care of the data in three column format

# now for 10y3mo file
df3 = pd.read_csv(data_folder / "data_10y3mo.csv")

In [37]:
# convert date to year and quarter
df3['year'] = pd.to_datetime(df3.date).dt.year
df3['qtr'] = pd.to_datetime(df3.date).dt.quarter

In [38]:
# need to find the mean rate for each quarter
mean_yield = df3.groupby(['year', 'qtr'])['10y3mo'].mean()

In [41]:
mean_yield.to_csv(r'data_yield_fixed.csv', header = "True")
# file renamed to data_yield_fixed.csv  adding legend row for year, qtr, yield

#### Merging all the files

# windows
df1new = pd.read_csv(data_folder / "data_fedrate_fixed.csv")
df2new = pd.read_csv(data_folder / "data_yield_fixed.csv")
df3new = pd.read_csv(data_folder / "data_bdi_fixed.csv")

In [43]:
# mac
df1new = pd.read_csv("data_fedrate_fixed.csv")
df2new = pd.read_csv("data_yield_fixed.csv")
df3new = pd.read_csv("data_bdi_fixed.csv")

In [44]:
df3new[:5]

,year,qtr,bdi,r
0,1985,1,978,0
1,1985,3,819,0
2,1985,4,820,0
3,1986,2,620,0
4,1986,4,748,0


In [45]:
# merging on year and qtr
merge1 = pd.merge(df1new, df2new, on=['year', 'qtr'], how = "left")

In [46]:
merge1[:10]

,year,qtr,fed_rate,10y3mo
0,1954,3,1.026667,NaN
1,1954,4,0.986667,NaN
2,1955,1,1.343333,NaN
3,1955,2,1.500000,NaN
4,1955,3,1.940000,NaN
5,1955,4,2.356667,NaN
6,1956,1,2.483333,NaN
7,1956,2,2.693333,NaN
8,1956,3,2.810000,NaN
9,1956,4,2.926667,NaN


In [47]:
# second merge on year and qtr
merge2 = pd.merge(merge1, df3new, on=['year', 'qtr'], how = "left")

In [48]:
merge2[:10]

,year,qtr,fed_rate,10y3mo,bdi,r
0,1954,3,1.026667,NaN,NaN,NaN
1,1954,4,0.986667,NaN,NaN,NaN
2,1955,1,1.343333,NaN,NaN,NaN
3,1955,2,1.500000,NaN,NaN,NaN
4,1955,3,1.940000,NaN,NaN,NaN
5,1955,4,2.356667,NaN,NaN,NaN
6,1956,1,2.483333,NaN,NaN,NaN
7,1956,2,2.693333,NaN,NaN,NaN
8,1956,3,2.810000,NaN,NaN,NaN
9,1956,4,2.926667,NaN,NaN,NaN


In [49]:
# need to declare a new df, or dropna doesn't work properly
dropna_merged_data = merge2.dropna(how='any')

In [50]:
dropna_merged_data[:10]

,year,qtr,fed_rate,10y3mo,bdi,r
122,1985,1,8.476667,3.080984,978.0,0.0
124,1985,3,7.900000,3.025323,819.0,0.0
125,1985,4,8.103333,2.760635,820.0,0.0
127,1986,2,6.920000,1.368197,620.0,0.0
129,1986,4,6.266667,1.780469,748.0,0.0
130,1987,1,6.220000,1.794754,945.0,0.0
132,1987,3,6.843333,2.426154,1042.0,0.0
133,1987,4,6.916667,2.867541,1299.0,0.0
134,1988,1,6.663333,2.404833,1409.0,0.0
136,1988,3,7.983333,1.984603,1291.0,0.0


In [51]:
dropna_merged_data.to_csv(r'data_all_quarterly.csv' ,index=False)